In [3]:
import numpy as np
import pandas as pd

import math
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import tensorflow_probability as tfp
from keras.models import Sequential
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import StringLookup
import joblib

#ignore warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
# brewer2mpl.get_map args: set name set type number of colors
import matplotlib as mpl

#mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

colors = ['#1E90FF', '#f12c60', '#fff21e', '#1efff2', '#1eafff', 'blueviolet', 'sandybrown', 'magenta', 'slategray']

warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)

# 1. Distance Result Plot

In [ ]:
# Distance result for quantile power curve
color_array = np.array([0.04771361, 0.04099092, 0.07114183, 0.04500382, 0.10449788,
       0.1041521 , 0.03867871, 0.04786069, 0.04113251, 0.10163694,
       0.07329147, 0.13186739, 0.04504443, 0.04294066, 0.05433169,
       0.05090413, 0.04146273, 0.11397119, 0.04067904, 0.0472645 ,
       0.07133181, 0.04302343, 0.05300268])

size_array = color_array*500

pyplt = py.offline.plot

trace0 = go.Scatter(
 x=[10, 20, 30, 40, 50, 60, 70, 80, 10, 20, 30, 40, 50, 60, 70, 80, 10, 20, 30, 40, 50, 60, 70],
 y=[50, 50, 50, 50, 50, 50, 50, 50, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10, 10, 10, 10, 10],
 mode='markers',
 marker=dict(
     color=color_array,
     size=size_array,
     showscale= True,
 )
)

layout = go.Layout(
    template = "plotly_white",
)

data = [trace0]
fig = go.Figure(data, layout=layout)
pyplt(fig, filename='plotly/quantile_622.html')

'plotly/quantile_622.html'

In [ ]:
# Distance result for density power curve
color_array = np.array([ 96.64900179,  27.58431625,  33.91370404,  27.27141006,
        60.692221  ,  87.60959361,  95.55038119,  65.13078653,
        34.3630244 ,  51.96083556,  36.06071791,  95.00573825,
        43.97924379,  29.50257021,  23.61064815,  39.10620063,
        32.13636829,  28.77216997, 124.58579613,  26.31951997,
        28.91708966,  37.11561012,  50.19967644])

size_array = color_array/2

pyplt = py.offline.plot

trace0 = go.Scatter(
 x=[10, 20, 30, 40, 50, 60, 70, 80, 10, 20, 30, 40, 50, 60, 70, 80, 10, 20, 30, 40, 50, 60, 70],
 y=[50, 50, 50, 50, 50, 50, 50, 50, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10, 10, 10, 10, 10],
 mode='markers',
 marker=dict(
     color=color_array,
     size=size_array,
     showscale= True,
 )
)

layout = go.Layout(
    template = "plotly_white",
)

data = [trace0]
fig = go.Figure(data, layout=layout)
pyplt(fig, filename='plotly/densisy_new.html')

'plotly/densisy_new.html'

# 2. Control chart plot

### 2.1 Comparison of KDE and BCKDE plot

In [5]:
import scipy

def calculate_cdf(dis, model, alpha):
  value_list = np.linspace(min(dis)-max(dis), 2*max(dis), 2000)

  for i in range(len(value_list)-1):
      value1 = model.integrate_box_1d(low = -np.Inf, high = value_list[i])
      value2 = model.integrate_box_1d(low = -np.Inf, high = value_list[i+1])
      if value1 <= alpha and value2 >= alpha:
          return (value_list[i] + value_list[i+1])/2

  return np.nan

In [ ]:
# data segment 1
# dis_array1:KDE for density; dis_array2: KDE for quantile
dis_array1 = np.array([96.64900179,  27.58431625,  33.91370404,  27.27141006,
        60.692221  ,  87.60959361,  95.55038119,  65.13078653,
        34.3630244 ,  51.96083556,  36.06071791,  95.00573825,
        43.97924379,  29.50257021,  23.61064815,  39.10620063,
        32.13636829,  28.77216997, 124.58579613,  26.31951997,
        28.91708966,  37.11561012,  50.19967644])

dis_array2 = np.array([0.04771361, 0.04099092, 0.07114183, 0.04500382, 0.10449788,
       0.1041521 , 0.03867871, 0.04786069, 0.04113251, 0.10163694,
       0.07329147, 0.13186739, 0.04504443, 0.04294066, 0.05433169,
       0.05090413, 0.04146273, 0.11397119, 0.04067904, 0.0472645 ,
       0.07133181, 0.04302343, 0.05300268])

# data segment 17
# dis_array1 = np.array([19.21823821,  26.37355637,  20.45925955,  13.21574669,
#         20.73780226,  16.16236866,  18.42269676,  17.27102678,
#         25.36348532,  70.13962352,  22.96070552,  21.93193337,
#         18.57146628,  26.33059136,  22.24590496,  52.64594953,
#         19.0951305 ,  23.39369544,  14.53528179,  12.61670661,
#        479.40089303,  21.74729134,  23.27001446])

# dis_array2 = np.array([0.04217547, 0.0306088 , 0.04511922, 0.03611985, 0.04528582,
#        0.03558038, 0.03903112, 0.03612916, 0.08610355, 0.07823464,
#        0.03827606, 0.03065592, 0.03588845, 0.04807117, 0.03217181,
#        0.05077566, 0.03152876, 0.07413133, 0.03110141, 0.0432993,
#        0.41819772, 0.03414809, 0.03817185])

In [ ]:
kde_model1 = scipy.stats.gaussian_kde(dis_array1, bw_method = 0.2)
cdf_list1 = [calculate_cdf(dis_array1, kde_model1, confidence) for confidence in np.linspace(0.01,0.99,99)]

kde_model2 = scipy.stats.gaussian_kde(dis_array2, bw_method = 0.2)
cdf_list2 = [calculate_cdf(dis_array2, kde_model2, confidence) for confidence in np.linspace(0.01,0.99,99)]

kde_model3 = scipy.stats.gaussian_kde(dis_array1, bw_method = 0.8)
cdf_list3 = [calculate_cdf(dis_array1, kde_model3, confidence) for confidence in np.linspace(0.01,0.99,99)]

kde_model4 = scipy.stats.gaussian_kde(dis_array2, bw_method = 0.8)
cdf_list4 = [calculate_cdf(dis_array2, kde_model4, confidence) for confidence in np.linspace(0.01,0.99,99)]

In [ ]:
# for data segment #1
df1 = pd.read_csv('cumulativeplot4bckde_bw0.2_2.csv',)
df_array1 = np.array(df1)

df2 = pd.read_csv('cumulativeplot4bckde_bw0.8_2.csv',)
df_array2 = np.array(df2)

# for data segment #17
# df1 = pd.read_csv('2_cumulativeplot4bckde_bw0.2.csv',)
# df_array1 = np.array(df1)

# df2 = pd.read_csv('2_cumulativeplot4bckde_bw0.8.csv',)
# df_array2 = np.array(df2)

KDE and BCKDE

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(25, 25), dpi=300, sharex=False, sharey=False)

ind = np.linspace(0.01,0.99,99).searchsorted(0.95)
alarm_boundary = cdf_list1[ind]

marker_list = ['o', '^', 's', 'x', 'D']
color_list = ['#90EE90', 'magenta', 'sandybrown', '#1E90FF', '#f12c60',]

# KDE for DPC bw=0.2
axes[0][0].plot(cdf_list1, np.linspace(0.01,0.99,99))
axes[0][0].axvline(alarm_boundary, label='UCL', color='r', linewidth=3,  linestyle='-')
for i in range(23):
  color_id = i//5
  marker_id = i%5
  axes[0][0].axvline(dis_array1[i], ymin=0.02, ymax=0.98, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.5,  marker=marker_list[marker_id], markersize=6)# marker

axes[0][0].set_xlabel('Power curve distance',fontsize=17)
axes[0][0].set_ylabel('Cumulative probability',fontsize=18)
axes[0][0].set_title('KDE for DPC (h=0.2)',fontsize=17)
axes[0][0].tick_params(labelsize=13)

# KDE for QPC bw=0.2
alarm_boundary = cdf_list2[ind]
axes[0][1].plot(cdf_list2, np.linspace(0.01,0.99,99))
axes[0][1].axvline(alarm_boundary, color='r', linewidth=3,  linestyle='-')
for i in range(23):
  color_id = i//5
  marker_id = i%5
  axes[0][1].axvline(dis_array2[i], ymin=0.02, ymax=0.98, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.5,  marker=marker_list[marker_id], markersize=6)

axes[0][1].set_xlabel('Power curve distance',fontsize=17)
axes[0][1].set_ylabel('Cumulative probability',fontsize=18)
axes[0][1].set_title('KDE for QPC (h=0.2)',fontsize=17)
axes[0][1].tick_params(labelsize=13)

# KDE for DPC bw=0.8
ind = np.linspace(0.01,0.99,99).searchsorted(0.95)
alarm_boundary = cdf_list3[ind]
axes[1][0].plot(cdf_list3, np.linspace(0.01,0.99,99))
axes[1][0].axvline(alarm_boundary, label='UCL', color='r', linewidth=3,  linestyle='-')
for i in range(23):
  color_id = i//5
  marker_id = i%5
  axes[1][0].axvline(dis_array1[i], ymin=0.02, ymax=0.98, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.5,  marker=marker_list[marker_id], markersize=6)

axes[1][0].set_xlabel('Power curve distance',fontsize=17)
axes[1][0].set_ylabel('Cumulative probability',fontsize=18)
axes[1][0].set_title('KDE for DPC (h=0.8)',fontsize=17)
axes[1][0].tick_params(labelsize=13)

# KDE for QPC bw=0.8
alarm_boundary = cdf_list4[ind]
axes[1][1].plot(cdf_list4, np.linspace(0.01,0.99,99))
axes[1][1].axvline(alarm_boundary, color='r', linewidth=3,  linestyle='-')
for i in range(23):
  color_id = i//5
  marker_id = i%5
  axes[1][1].axvline(dis_array2[i], ymin=0.02, ymax=0.98, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.5,  marker=marker_list[marker_id], markersize=6)

axes[1][1].set_xlabel('Power curve distance',fontsize=17)
axes[1][1].set_ylabel('Cumulative probability',fontsize=18)
axes[1][1].set_title('KDE for QPC (h=0.8)',fontsize=17)
axes[1][1].tick_params(labelsize=13)

# BCKDE for DPC bw=0.2
ind = df_array1[:,0].searchsorted(0.95)
alarm_boundary = np.linspace(0, np.max(dis_array1)+100, 1000)[ind]
axes[2][0].plot(np.linspace(0, np.max(dis_array1)+100, 1000), df_array1[:,0])
axes[2][0].axvline(alarm_boundary, color='r', linewidth=3,  linestyle='-')
for i in range(23):
  color_id = i//5
  marker_id = i%5
  axes[2][0].axvline(dis_array1[i], ymin=0.02, ymax=0.98, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.5,  marker=marker_list[marker_id], markersize=6)

axes[2][0].set_xlabel('Power curve distance',fontsize=17)
axes[2][0].set_ylabel('Cumulative probability',fontsize=18)
axes[2][0].set_title('BCKDE for DPC (h=0.2)',fontsize=17)
axes[2][0].tick_params(labelsize=13)

# BCKDE for QPC bw=0.2
ind = df_array1[:,1].searchsorted(0.95)
alarm_boundary = np.linspace(0, np.max(dis_array2)+0.1, 1000)[ind]
axes[2][1].plot(np.linspace(0, np.max(dis_array2)+0.1, 1000), df_array1[:,1])
axes[2][1].axvline(alarm_boundary, color='r', linewidth=3,  linestyle='-')
for i in range(23):
  color_id = i//5
  marker_id = i%5
  axes[2][1].axvline(dis_array2[i], ymin=0.02, ymax=0.98, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.5,  marker=marker_list[marker_id], markersize=6)

axes[2][1].set_xlabel('Power curve distance',fontsize=17)
axes[2][1].set_ylabel('Cumulative probability',fontsize=18)
axes[2][1].set_title('BCKDE for QPC (h=0.2)',fontsize=17)
axes[2][1].tick_params(labelsize=13)

# BCKDE for DPC bw=0.8
ind = df_array2[:,0].searchsorted(0.95)
alarm_boundary = np.linspace(0, np.max(dis_array1)+100, 1000)[ind]
axes[3][0].plot(np.linspace(0, np.max(dis_array1)+100, 1000), df_array2[:,0])
axes[3][0].axvline(alarm_boundary, color='r', linewidth=3,  linestyle='-')
for i in range(23):
  color_id = i//5
  marker_id = i%5
  axes[3][0].axvline(dis_array1[i], ymin=0.02, ymax=0.98, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.5,  marker=marker_list[marker_id], markersize=6)

axes[3][0].set_xlabel('Power curve distance',fontsize=17)
axes[3][0].set_ylabel('Cumulative probability',fontsize=18)
axes[3][0].set_title('BCKDE for DPC (h=0.8)',fontsize=17)
axes[3][0].tick_params(labelsize=13)

# BCKDE for QPC bw=0.8
ind = df_array2[:,1].searchsorted(0.95)
alarm_boundary = np.linspace(0, np.max(dis_array2)+0.1, 1000)[ind]
axes[3][1].plot(np.linspace(0, np.max(dis_array2)+0.1, 1000), df_array2[:,1])
axes[3][1].axvline(alarm_boundary, color='r', linewidth=3,  linestyle='-')
for i in range(23):
  color_id = i//5
  marker_id = i%5
  axes[3][1].axvline(dis_array2[i], ymin=0.02, ymax=0.98, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.5,  marker=marker_list[marker_id], markersize=6)

axes[3][1].set_xlabel('Power curve distance',fontsize=17)
axes[3][1].set_ylabel('Cumulative probability',fontsize=18)
axes[3][1].set_title('BCKDE for QPC (h=0.8)',fontsize=17)
axes[0][0].legend(bbox_to_anchor=(0, 1.1), loc=3, borderaxespad=0.1, ncol=8, fontsize=20)

axes[3][1].tick_params(labelsize=13)

BCKDE only

In [ ]:
# BCKDE for DPC bw=0.2
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 10), dpi=300, sharex=False, sharey=False)

ind = np.linspace(0.01,0.99,99).searchsorted(0.95)
marker_list = ['o', '^', 's', 'x', 'D']
color_list = ['#90EE90', 'magenta', 'sandybrown', '#1E90FF', '#f12c60',]
ind = df_array1[:,0].searchsorted(0.95)

alarm_boundary = np.linspace(0, np.max(dis_array1)+100, 1000)[ind]
axes[0].plot(np.linspace(0, np.max(dis_array1)+100, 1000), df_array2[:,0])
axes[0].axvline(alarm_boundary, color='r', label='UCL', linewidth=2,  linestyle='-')
for i in range(23):
  color_id = i//5
  marker_id = i%5
  axes[0].axvline(dis_array1[i], ymin=0.01, ymax=0.99, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.3,  marker=marker_list[marker_id], markersize=5)

axes[0].set_xlabel('Power curve distance',fontsize=13)
axes[0].set_ylabel('Cumulative probability',fontsize=13)
axes[0].set_title('BCKDE for DPC (h=0.8)',fontsize=13)

# BCKDE for QPC bw=0.2
ind = df_array1[:,1].searchsorted(0.95)
alarm_boundary = np.linspace(0, np.max(dis_array2)+0.1, 1000)[ind]
axes[1].plot(np.linspace(0, np.max(dis_array2)+0.1, 1000), df_array2[:,1])
axes[1].axvline(alarm_boundary, color='r', linewidth=2,  linestyle='-')
for i in range(23):
  color_id = i//5
  marker_id = i%5
  axes[1].axvline(dis_array2[i], ymin=0.01, ymax=0.99, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.3,  marker=marker_list[marker_id], markersize=5)

axes[1].set_xlabel('Power curve distance',fontsize=13)
axes[1].set_ylabel('Cumulative probability',fontsize=13)
axes[1].set_title('BCKDE for QPC (h=0.8)',fontsize=13)
axes[0].legend(bbox_to_anchor=(0, 1.1), loc=3, borderaxespad=0.1, ncol=6, fontsize=13)

In [ ]:
# Generate simulation data for BCKDE Cumulative plot
df = pd.DataFrame()
df['V1'] = np.linspace(0, np.max(dis_array1)+100, 1000)
df['V2'] = np.linspace(0, np.max(dis_array2)+0.1, 1000)
df.to_csv('2_data4BCKDECumulative.csv', index=False)

# Record distance value
df = pd.DataFrame()
df['V1'] = dis_array1
df['V2'] = dis_array2
df.to_csv('2_distance4BCKDECumulative.csv', index=False)

### 2.2 Distance matrix plot

#### 2.2.1 QPC

In [ ]:
import scipy

def calculate_cdf(dis, model, alpha):
  value_list = np.linspace(min(dis)-max(dis), 2*max(dis), 2000)

  for i in range(len(value_list)-1):
      value1 = model.integrate_box_1d(low = -np.Inf, high = value_list[i])
      value2 = model.integrate_box_1d(low = -np.Inf, high = value_list[i+1])
      if value1 <= alpha and value2 >= alpha:
          return (value_list[i] + value_list[i+1])/2

  return np.nan

In [ ]:
distance_matrix = np.load('distance_matrix/QPC.npy')
distance_matrix = distance_matrix[:33]

In [ ]:
# KDE
from tqdm import tqdm

ind = np.linspace(0.01,0.99,99).searchsorted(0.95)

UCL_list = []
for i in tqdm(range(33)):
  kde_model = scipy.stats.gaussian_kde(distance_matrix[i], bw_method = 0.5)
  cdf_list = [calculate_cdf(distance_matrix[i], kde_model, confidence) for confidence in np.linspace(0.01,0.99,99)]
  UCL_list.append(cdf_list[ind])

100%|██████████| 33/33 [02:02<00:00,  3.71s/it]


In [ ]:
predict_matrix = np.zeros((distance_matrix.shape[0], distance_matrix.shape[1]))

for i in range(len(distance_matrix)):
    for j in range(len(distance_matrix[i])):
        if distance_matrix[i][j] > UCL_list[i]:
            predict_matrix[i][j] = 1

In [ ]:
predict_matrix

In [ ]:
# BCKDE
data4qpc = pd.read_csv('distance_matrix/controlchart4qpc.csv').to_numpy()

UCL_list_bckde = []
for i in tqdm(range(33)):
  ind = data4qpc[:,i].searchsorted(0.95)
  UCL_list_bckde.append(np.linspace(0, max(distance_matrix[i])+0.1, 1000)[ind])

100%|██████████| 33/33 [00:00<00:00, 13497.03it/s]


In [ ]:
data4qpc = pd.read_csv('distance_matrix/controlchart4qpc.csv').to_numpy()

In [ ]:
from matplotlib.pyplot import MultipleLocator

plt.figure(figsize=(8, 5))
marker_list = ['o', '^', 's', 'x', 'D']
color_list = ['#90EE90', 'magenta', 'sandybrown', '#1E90FF', 'darkorchid',] ##f12c60

fig, axes = plt.subplots(figsize=(12.3, 6), dpi=300)

# plt.plot(UCL_list, label='UCL of KDE', c='red', linestyle='--', linewidth=2,) # marker='s',markersize=5,
plt.plot(UCL_list_bckde, label='UCL of BCKDE', c='red', linewidth=2,) # marker='o', linestyle='--', markersize=5,

index_array = np.array([6, 13, 14, 19, 20])
for i in range(len(index_array)):
  color_id = i
  # color_id = i//5
  # marker_id = i%5
  plt.plot(distance_matrix[:,index_array[i]], label='WT #%s'%(index_array[i]+1), c=color_list[color_id], linewidth=1,linestyle='--') # marker=marker_list[marker_id], markersize=3,

plt.xlabel('Time Index', fontsize=16)
plt.ylabel('Power curve distance', fontsize=16)
plt.legend(bbox_to_anchor=(0, 1), loc=3, borderaxespad=0.1, ncol=6) #ncol=7

x_major_locator=MultipleLocator(1)
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.xlim(-0.2,32.2)

#### 2.2.2 DPC

In [ ]:
distance_matrix = np.load('distance_matrix/DPC.npy')

In [ ]:
# KDE

from tqdm import tqdm

ind = np.linspace(0.01,0.99,99).searchsorted(0.95)

UCL_list = []
for i in tqdm(range(33)):
  kde_model = scipy.stats.gaussian_kde(distance_matrix[i], bw_method = 0.5)
  cdf_list = [calculate_cdf(distance_matrix[i], kde_model, confidence) for confidence in np.linspace(0.01,0.99,99)]
  UCL_list.append(cdf_list[ind])

distance_matrix = distance_matrix[:33]

100%|██████████| 33/33 [02:10<00:00,  3.95s/it]


In [ ]:
# BCKDE
data4qpc = pd.read_csv('distance_matrix/controlchart4dpc.csv').to_numpy()

UCL_list_bckde = []
for i in tqdm(range(33)):
  ind = data4qpc[:,i].searchsorted(0.95)
  UCL_list_bckde.append(np.linspace(0, max(distance_matrix[i])+100, 1000)[ind])

100%|██████████| 33/33 [00:00<00:00, 12689.04it/s]


In [ ]:
from matplotlib.pyplot import MultipleLocator

plt.figure(figsize=(8, 5))
marker_list = ['o', '^', 's', 'x', 'D']
color_list = ['#90EE90', 'magenta', 'sandybrown', '#1E90FF', 'darkorchid',]

fig, axes = plt.subplots(figsize=(12.3, 6), dpi=300)

# plt.plot(UCL_list, label='UCL of KDE', c='red', linestyle='--', linewidth=2,) # marker='s',markersize=5,
plt.plot(UCL_list_bckde, label='UCL of BCKDE', c='red', linewidth=2,) # marker='o', linestyle='--', markersize=5,

# for i in range(23):
#   color_id = i//5
#   marker_id = i%5
#   plt.plot(distance_matrix[:,i], label='WT #%s'%(i+1), c=color_list[color_id], marker=marker_list[marker_id], markersize=3, linewidth=1,linestyle='--')
index_array = np.array([6, 13, 14, 19, 20])
for i in range(len(index_array)):
  color_id = i
  # color_id = i//5
  # marker_id = i%5
  plt.plot(distance_matrix[:,index_array[i]], label='WT #%s'%(index_array[i]+1), c=color_list[color_id], linewidth=1,linestyle='--')

plt.xlabel('Time Index', fontsize=16)
plt.ylabel('Power curve distance', fontsize=16)
# plt.legend(bbox_to_anchor=(0, 1), loc=3, borderaxespad=0.1, ncol=7)

x_major_locator=MultipleLocator(1)
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.xlim(-0.2,32.2)

#### 2.2.2 BCKDE

In [ ]:
"""
# for QPC, +0.1; for DPC, +100
distance_matrix = np.load('distance_matrix/QPC.npy')[:33]
simulation4qpc = np.zeros((1000,33))
for i in range(33):
  simulation4qpc[:,i] = np.linspace(0, max(distance_matrix[i])+0.1, 1000)

df = pd.DataFrame(data=simulation4qpc)
df.to_csv('distance_matrix/simulation4qpc.csv',index=False)

distance_matrix = np.load('distance_matrix/QPC.npy')[:33].T
df = pd.DataFrame(data=distance_matrix)
df.to_csv('distance_matrix/distance4qpc.csv',index=False)

distance_matrix = np.load('distance_matrix/DPC.npy')[:33]
simulation4dpc = np.zeros((1000,33))
for i in range(33):
  simulation4dpc[:,i] = np.linspace(0, max(distance_matrix[i])+100, 1000)

df = pd.DataFrame(data=simulation4dpc)
df.to_csv('distance_matrix/simulation4dpc.csv',index=False)

distance_matrix = np.load('distance_matrix/DPC.npy')[:33].T
df = pd.DataFrame(data=distance_matrix)
df.to_csv('distance_matrix/distance4dpc.csv',index=False)
"""

BCKDE for QPC

### 2.1 Scatter of label

In [ ]:
label_set = []
window = 1000

for i in range(len(nrm)):
  nrm0 = nrm[i]
  label0 = nrm0[:, 2::3]
  label_set.append(np.sum(label0, axis=0)/window)

In [ ]:
label_set = np.array(label_set)

In [ ]:
np.save('distance_matrix/label.npy', label_set)

In [ ]:
from matplotlib.pyplot import MultipleLocator

plt.figure(figsize=(12, 6))
marker_list = ['o', '^', 's', 'x', 'D']
color_list = ['#90EE90', 'magenta', 'sandybrown', '#1E90FF', '#f12c60',]

fig, axes = plt.subplots(figsize=(12.3, 6), dpi=300)

for i in range(23):
  color_id = i//5
  marker_id = i%5
  plt.plot(label_set[:,i][:33], label='WT #%s'%(i+1), c=color_list[color_id], marker=marker_list[marker_id], markersize=3)

axes.axhline(0.6, label='Threshold', color='gray', linewidth=2,  linestyle='--')
plt.xlabel('Time Index', fontsize=16)
plt.ylabel('Degradation Index', fontsize=16)
plt.legend(bbox_to_anchor=(0, 1), loc=3, borderaxespad=0.1, ncol=8)

x_major_locator=MultipleLocator(1)
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.xlim(-0.2,32.2)

# 5. Relative degraded variable plot

Observe the variable changes of WT # 22 from 2019-02-27 15:10:00 to 2019-04-15 10:20:00; And compared with normal wind turbine

In [ ]:
variable_df = pd.read_csv('variable.csv')

turbine_names = []
for _ in range(49619):
  for i in range(25):
    turbine_names.append('风机_%s'%i)

variable_df['风机名称'] = turbine_names

# 4. Result for power curve construction

Result for Quantile Power Curve

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model
from keras.utils.vis_utils import plot_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform,RandomNormal
from matplotlib import pyplot as plt

tfd = tfp.distributions

In [ ]:
def qloss(y_true, y_pred, n_q=99):
    q = np.array(range(1, n_q + 1))
    left = (q / (n_q + 1) - 1) * (y_true - y_pred)
    right = q / (n_q + 1) * (y_true - y_pred)
    return K.mean(K.maximum(left, right))

from keras.callbacks import *

In [ ]:
def QRNN(num_quantiles):
  mean_inputs = tf.keras.Input(1)
  mean_x = Dense(256,)(mean_inputs)
  mean_x=Dense(256,)(mean_x)
  mean_x = Dense(128,)(mean_x)
  mean_x = Dense(64,)(mean_x)
  mean_x = Dense(32,)(mean_x)
  outputs=Dense(num_quantiles)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

def QRNN_2(num_quantiles):
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(64, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  outputs=Dense(num_quantiles)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

In [ ]:
def find_nearest(a, a0):
    "Element in nd array `a` closest to the scalar value `a0`"
    idx = np.abs(a - a0).argmin()
    return idx

In [ ]:
import matplotlib as mpl

#mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

In [ ]:
i = 0

nrm0 = nrm[i]
speed0 = nrm0[:, 0::3]
power0 = nrm0[:, 1::3]
label0 = nrm0[:, 2::3]

speed = nrm0[:, 0::3]
power = nrm0[:, 1::3]
omega = 0.2
std = np.std(speed0[:,0])
speed_new = [[] for _ in range(23)]
power_new = [[] for _ in range(23)]
speed_new[0] = speed[:,0]
power_new[0] = power[:,0]

for j in range(1, 23):
  for t in range(len(speed[:,0])):
    idx = find_nearest(speed[:,j], speed[:,0][t])
    tmp = speed[:,j].flat[idx]
    if abs(tmp - speed[:,0][t]) <  omega*std:
      speed_new[j].append(tmp)
      power_new[j].append(power[:,j].flat[idx])


m, n = 0, 0
fig, axes = plt.subplots(6, 4, figsize=(20, 25), dpi=300, sharex=True, sharey=True)

colors = ['#1efff2', '#f12c60', 'magenta', '#1efff2', '#1eafff', 'blueviolet', 'sandybrown', 'magenta', '#f12c60']

for k in range(23):
  print("Current segment: %s; WT: %s"%(i, k))
  speed_list = np.array(speed_new[k])
  speed_list = speed_list.reshape(len(speed_list), 1)
  power_list = np.array(power_new[k])

  power_list = (power_list - 0)/4000 # set between 0 and 1
  power_list[power_list>1] = 1
  power_list[power_list<0] = 0

  num_quantiles = 9
  model = QRNN_2(num_quantiles)

  # Train
  model.load_weights("model_quantile2/model_%s_%s.h5"%(i,k))

  test_pred = model.predict(speed_list)

  axes[m][n].scatter(speed_list, power_list , s=7, c='darkgray', label = 'Actual data')


  for d in range(0, num_quantiles, 2):
    preds = test_pred[:, d]
    preds[preds < 0] = 0
    value = (d + 1)*10
    axes[m][n].scatter(speed_list, preds, c=colors[d], label = "Quantile level =  %s%%"%str(value), s=1)


  axes[m][n].set_title('WT #%s'%(k+1), fontsize=17)
  axes[m][n].set_xlim(2,13)
  axes[m][n].set_ylim(-0.05,1.05)
  axes[m][n].tick_params(labelsize=17)
  if n==0:
    axes[m][n].set_ylabel('Wind Power', fontsize=17)

  if m==5:
    axes[m][n].set_xlabel('Wind Speed', fontsize=17)

  n += 1
  if n >3:
    m += 1
    n = 0

axes[5][3].set_xlabel('Wind Speed', fontsize=17)
axes[5][3].tick_params(labelsize=17)
lgd = axes[0][0].legend(bbox_to_anchor=(0, 1.2), loc=3, borderaxespad=0.1, ncol=3, fontsize=20)
for legend_handle in lgd.legendHandles:
    legend_handle.set_sizes([30])

Result for Density Power Curve

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model
from keras.utils.vis_utils import plot_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform,RandomNormal
from matplotlib import pyplot as plt

from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold, cross_val_score, train_test_split

tfd = tfp.distributions

In [ ]:
def BNN_normal():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x = Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  # outputs = Dense(1)(mean_x)
  model = keras.Model(inputs=[mean_inputs], outputs=[outputs])
  return model

In [ ]:
from scipy.stats import norm
import matplotlib.colors as mcolors
import random

i = 0

nrm0 = nrm[i]
speed0 = nrm0[:, 0::3]
power0 = nrm0[:, 1::3]
label0 = nrm0[:, 2::3]

speed = nrm0[:, 0::3]
power = nrm0[:, 1::3]
omega = 0.2
std = np.std(speed0[:,0])
speed_new = [[] for _ in range(23)]
power_new = [[] for _ in range(23)]
speed_new[0] = speed[:,0]
power_new[0] = power[:,0]

for j in range(1, 23):
  for t in range(len(speed[:,0])):
    idx = find_nearest(speed[:,j], speed[:,0][t])
    tmp = speed[:,j].flat[idx]
    if abs(tmp - speed[:,0][t]) <  omega*std:
      speed_new[j].append(tmp)
      power_new[j].append(power[:,j].flat[idx])

colors = ['saddlebrown', 'springgreen', 'blueviolet', 'sandybrown', 'deeppink', 'darkviolet']

m, n = 0, 0
fig, axes = plt.subplots(6, 4, figsize=(20, 25), dpi=300, sharex=True, sharey=True)

for k in range(23):

  print("Current segment: %s; WT: %s"%(i, k))
  speed_list = np.array(speed_new[k])
  speed_list = speed_list.reshape(len(speed_list), 1)
  power_list = np.array(power_new[k])

  power_list = (power_list - 0)/4000 # set between 0 and 1
  power_list[power_list>1] = 1
  power_list[power_list<0] = 0

  model = BNN_normal()

  # Train
  model.load_weights("model_densitylstm/model_%s_%s.h5"%(i,k))

  test_pred = model(speed_list)
  axes[m][n].scatter(speed_list, power_list , s=3, c='darkgray', label = 'Actual data')

  seed = 2
  # random.seed(seed)
  input_list = np.linspace(3, 13, 12)

  step = 0
  for p in range(0, len(input_list), 2):
    test_pred = model(np.array(input_list[p]).reshape(1,1,1))

    prediction_mean = test_pred.mean().numpy().tolist()[0]
    prediction_stdv = test_pred.stddev().numpy()[0]

    prediction_mean = np.array(prediction_mean)
    prediction_mean[prediction_mean<0] = 0
    prediction_mean[prediction_mean>1] = 1

    x = np.linspace(prediction_mean-2*prediction_stdv, prediction_mean+2*prediction_stdv, 200)

    res = norm.pdf(x, loc=prediction_mean, scale=prediction_stdv)
    res = (res - min(res))/(1.5*(max(res) - min(res)))

    axes[m][n].plot(input_list[p]+res, x , linewidth=1.5, color=colors[step], label='Speed = %sm/s'%round(input_list[p],2), linestyle='--')
    step += 1

  prediction = model(np.array(input_list).reshape(12 ,1,1))
  prediction_mean = prediction.mean().numpy()
  prediction_mean[prediction_mean<0] = 0
  prediction_mean[prediction_mean>1] = 1
  axes[m][n].plot(input_list, prediction_mean, linewidth=2, c='#f12c60', label='Prediction mean')
  axes[m][n].set_xlim(2,13)
  axes[m][n].set_ylim(-0.05,1.05)

  axes[m][n].set_title('WT #%s'%(k+1), fontsize=15)

  if n==0:
    axes[m][n].set_ylabel('Wind Power', fontsize=15)

  if m==5:
    axes[m][n].set_xlabel('Wind Speed', fontsize=15)

  n += 1
  if n >3:
    m += 1
    n = 0

axes[5][3].set_xlabel('Wind Speed', fontsize=15)
axes[0][0].legend(bbox_to_anchor=(0, 1.2), loc=3, borderaxespad=0.1, ncol=4, fontsize=20)

In [ ]:
seed = 2
random.seed(seed)
colors = list(mcolors.CSS4_COLORS.keys())
random.shuffle(colors)

In [ ]:
from scipy.stats import norm
import matplotlib.colors as mcolors
import random

seed = 2
random.seed(seed)

input_list = np.linspace(3, 18, 16)
fig, axes = plt.subplots(4, 6, figsize=(25, 20), dpi=300, sharex=True, sharey=True)

axes[0][0].scatter(speed_list, power_list , s=3, c='darkgray', label = 'Actual data')


step = 0
for i in range(len(input_list)):
  test_pred = model(np.array(input_list[i]).reshape(1,1,1))

  prediction_mean = test_pred.mean().numpy().tolist()[0]
  prediction_stdv = test_pred.stddev().numpy()[0]

  x = np.linspace(prediction_mean-2*prediction_stdv, prediction_mean+2*prediction_stdv, 200)

  res = norm.pdf(x, loc=prediction_mean, scale=prediction_stdv)
  res = (res - min(res))/(1.5*(max(res) - min(res)))

  axes[0][0].plot(input_list[i]+res, x , linewidth=2, color=colors[step], label='Speed = %sm/s'%input_list[i])
  step += 1

prediction = model(np.array(input_list).reshape(16 ,1,1))
prediction_mean = prediction.mean().numpy().tolist()

axes[0][0].plot(input_list, prediction_mean, linewidth=2, c='#f12c60', label='Prediction mean')
axes[0][0].set_xlim(2,19)
axes[0][0].legend(bbox_to_anchor=(0, 1.2), loc=3, borderaxespad=0.1, ncol=6, fontsize=15)

# 5. Distance matrix historgam visualization

In [ ]:
data = np.load('distance_matrix/distance_DPC.npy')

In [ ]:
from scipy.stats import norm
import matplotlib.colors as mcolors
import random

m, n = 0, 0
fig, axes = plt.subplots(4, 6, figsize=(25, 20), dpi=300, sharex=False, sharey=False)

for k in range(23):

  axes[m][n].hist(data[:,k], color='#1eafff')
  axes[m][n].set_title('WT #%s'%(k+1), fontsize=15)

  if n==0:
    axes[m][n].set_ylabel('Count', fontsize=15)

  if m==3:
    axes[m][n].set_xlabel('Power Curve Distance', fontsize=15)

  n += 1
  if n >5:
    m += 1
    n = 0

axes[2][5].set_xlabel('Power Curve Distance', fontsize=15)

In [ ]:
plt.hist(data[1,:], color='#1eafff')

# 6. one wind turbine; multiple data segment

In [ ]:
np.shape(speed[:,11])

(1000,)

In [ ]:
from scipy.stats import norm
import matplotlib.colors as mcolors
import random

m, n = 0, 0
fig, axes = plt.subplots(4, 6, figsize=(25, 20), dpi=300, sharex=False, sharey=False)

colors = ['saddlebrown', 'springgreen', 'blueviolet', 'sandybrown', 'deeppink', 'darkviolet']

for k in range(24):
  nrm0 = nrm[k]
  speed = nrm0[:, 0::3]
  power = nrm0[:, 1::3]

  print("Current segment: %s; WT: %s"%(i, k))
  speed_list = np.array(speed[:,11])
  speed_list = speed_list.reshape(len(speed_list), 1)
  power_list = np.array(power[:,11])

  power_list = (power_list - 0)/4000 # set between 0 and 1
  power_list[power_list>1] = 1
  power_list[power_list<0] = 0

  model = BNN_normal()

  # Train
  model.load_weights("model_densitylstm/model_%s_%s.h5"%(k,11))

  test_pred = model(speed_list)
  axes[m][n].scatter(speed_list, power_list , s=3, c='darkgray', label = 'Actual data')

  # seed = 2
  # random.seed(seed)
  input_list = np.linspace(np.min(speed), np.max(speed), 12)

  # colors = list(mcolors.CSS4_COLORS.keys())
  # random.shuffle(colors)

  step = 0
  for p in range(0, len(input_list), 2):
    test_pred = model(np.array(input_list[p]).reshape(1,1,1))

    prediction_mean = test_pred.mean().numpy().tolist()[0]
    prediction_stdv = test_pred.stddev().numpy()[0]
    prediction_mean = np.array(prediction_mean)
    prediction_mean[prediction_mean<0] = 0
    prediction_mean[prediction_mean>1] = 1
    x = np.linspace(prediction_mean-2*prediction_stdv, prediction_mean+2*prediction_stdv, 200)

    res = norm.pdf(x, loc=prediction_mean, scale=prediction_stdv)
    res = (res - min(res))/(1.5*(max(res) - min(res)))

    axes[m][n].plot(input_list[p]+res, x , linewidth=1.5, color=colors[step], label='Speed = %sm/s'%input_list[p], linestyle='--')
    step += 1

  prediction = model(np.array(input_list).reshape(12 ,1,1))
  prediction_mean = prediction.mean().numpy()
  prediction_mean[prediction_mean<0] = 0
  prediction_mean[prediction_mean>1] = 1
  axes[m][n].plot(input_list, prediction_mean, linewidth=2, c='#f12c60', label='Prediction mean')
  axes[m][n].set_ylim(-0.05,1.05)

  axes[m][n].set_title('Segment #%s'%(k+1), fontsize=15)

  if n==0:
    axes[m][n].set_ylabel('Wind Power', fontsize=15)

  if m==3:
    axes[m][n].set_xlabel('Wind Speed', fontsize=15)

  n += 1
  if n >5:
    m += 1
    n = 0

# axes[0][0].legend(bbox_to_anchor=(0, 1.2), loc=3, borderaxespad=0.1, ncol=6, fontsize=15)

# 7. Results of change Nu

In [ ]:
def BNN_normal():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x = Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  # outputs = Dense(1)(mean_x)
  model = keras.Model(inputs=[mean_inputs], outputs=[outputs])
  return model

In [ ]:
i = 0

nrm0 = nrm[i]
speed0 = nrm0[:, 0::3]
power0 = nrm0[:, 1::3]
label0 = nrm0[:, 2::3]

In [ ]:
from scipy.stats import norm
import matplotlib.colors as mcolors
import random

def find_nearest(a, a0):
    "Element in nd array `a` closest to the scalar value `a0`"
    idx = np.abs(a - a0).argmin()
    return idx

i = 0

nrm0 = nrm[i]
speed0 = nrm0[:, 0::3]
power0 = nrm0[:, 1::3]
label0 = nrm0[:, 2::3]

speed = nrm0[:, 0::3]
power = nrm0[:, 1::3]
omega = 0.2
std = np.std(speed0[:,0])
speed_new = [[] for _ in range(23)]
power_new = [[] for _ in range(23)]
speed_new[0] = speed[:,0]
power_new[0] = power[:,0]

for j in range(1, 23):
  for t in range(len(speed[:,0])):
    idx = find_nearest(speed[:,j], speed[:,0][t])
    tmp = speed[:,j].flat[idx]
    if abs(tmp - speed[:,0][t]) <  omega*std:
      speed_new[j].append(tmp)
      power_new[j].append(power[:,j].flat[idx])

m, n = 0, 0
fig, axes = plt.subplots(2, 6, figsize=(25, 10), dpi=300, sharex=True, sharey=True)

colors = ['saddlebrown', 'springgreen', 'blueviolet', 'sandybrown', 'deeppink', 'darkviolet']

for k in range(12):

  print("Current segment: %s; WT: %s"%(i, k))
  speed_list = np.array(speed_new[k])
  speed_list = speed_list.reshape(len(speed_list), 1)
  power_list = np.array(power_new[k])

  power_list = (power_list - 0)/4000 # set between 0 and 1
  power_list[power_list>1] = 1
  power_list[power_list<0] = 0

  model = BNN_normal()

  # Train
  # model.load_weights("model_density_nu/model_%s_%s_4000.h5"%(i,k))
  model.load_weights("model_densitylstm/model_%s_%s.h5"%(i,k))

  test_pred = model(speed_list)
  axes[m][n].scatter(speed_list, power_list , s=3, c='darkgray', label = 'Actual data')

  # seed = 2
  # random.seed(seed)
  input_list = np.linspace(2, 12, 12)

  # colors = list(mcolors.CSS4_COLORS.keys())
  # random.shuffle(colors)

  step = 0
  for p in range(0, len(input_list), 2):
    test_pred = model(np.array(input_list[p]).reshape(1,1,1))

    prediction_mean = test_pred.mean().numpy().tolist()[0]
    prediction_stdv = test_pred.stddev().numpy()[0]

    prediction_mean = np.array(prediction_mean)
    prediction_mean[prediction_mean<0] = 0
    prediction_mean[prediction_mean>1] = 1

    x = np.linspace(prediction_mean-2*prediction_stdv, prediction_mean+2*prediction_stdv, 200)

    res = norm.pdf(x, loc=prediction_mean, scale=prediction_stdv)
    res = (res - min(res))/(1.5*(max(res) - min(res)))

    axes[m][n].plot(input_list[p]+res, x , linewidth=1.5, color=colors[step], label='Speed = %sm/s'%round(input_list[p], 2), linestyle='--')
    step += 1

  prediction = model(np.array(input_list).reshape(12 ,1,1))
  prediction_mean = prediction.mean().numpy()
  prediction_mean[prediction_mean<0] = 0
  prediction_mean[prediction_mean>1] = 1

  axes[m][n].plot(input_list, prediction_mean, linewidth=2, c='#f12c60', label='Prediction mean')
  axes[m][n].set_xlim(2, 12)
  axes[m][n].set_ylim(-0.05,1.05)

  axes[m][n].set_title('WT #%s'%(k+1), fontsize=18)
  axes[m][n].tick_params(labelsize=18)

  if n==0:
    axes[m][n].set_ylabel('Wind Power', fontsize=18)

  if m==1:
    axes[m][n].set_xlabel('Wind Speed', fontsize=18)

  n += 1
  if n >5:
    m += 1
    n = 0

axes[1][5].set_xlabel('Wind Speed', fontsize=18)
axes[0][0].legend(bbox_to_anchor=(0, 1.2), loc=3, borderaxespad=0.1, ncol=4, fontsize=23)

# 8. comparison of two power curves

In [ ]:
from scipy.stats import norm
import scipy
import numpy as np
from numba import jit,njit
from tqdm import *

def calculate_kl_divergence(dis1, dis2):
    kl_div = 0
    min_1 = dis1[0] - 2*dis1[1]
    min_2 = dis2[0] - 2*dis2[1]
    max_1 = dis1[0] + 2*dis1[1]
    max_2 = dis2[0] + 2*dis2[1]
    min_value = min(min_1, min_2)
    max_value = max(max_1, max_2)
    values = np.linspace(min_value, max_value, 20)

    for x in values:
        px = norm.pdf(x, loc = dis1[0], scale = dis1[1])
        qx = norm.pdf(x, loc = dis2[0], scale = dis2[1])
        if px < 1e-16:
            px = 1e-16

        if qx < 1e-16:
            qx = 1e-16

        kl_div += px*np.log(px/qx)

    return kl_div

def calculate_quantile_distance(quantile_list1, quantile_list2):
  return np.mean((quantile_list1-quantile_list2)**2)

In [ ]:
from scipy.stats import norm
import matplotlib.colors as mcolors
import random

input_list = np.linspace(3, 13, 50)
input_list = input_list.reshape(len(input_list), 1)
i= 0
m = 1
n = 2
p = 22

# DPC model
model1 = BNN_normal()
model2 = BNN_normal()
model3 = BNN_normal()

model1.load_weights("model_densitylstm/model_%s_%s.h5"%(i,m))
model2.load_weights("model_densitylstm/model_%s_%s.h5"%(i,n))
model3.load_weights("model_densitylstm/model_%s_%s.h5"%(i,p))

test_pred1 = model1(input_list)
test_pred2 = model2(input_list)
test_pred3 = model3(input_list)

# QPC model
num_quantiles = 9
model4 = QRNN_2(num_quantiles)
model5 = QRNN_2(num_quantiles)
model6 = QRNN_2(num_quantiles)
model4.load_weights("model_quantile2/model_%s_%s.h5"%(i,m))
model5.load_weights("model_quantile2/model_%s_%s.h5"%(i,n))
model6.load_weights("model_quantile2/model_%s_%s.h5"%(i,p))

test_pred4 = model4.predict(input_list)
test_pred5 = model5.predict(input_list)
test_pred6 = model6.predict(input_list)

2/2 [==============================] - 1s 7ms/step


In [ ]:
test_pred4[0]

array([0.00126876, 0.00289074, 0.00573064, 0.00658844, 0.01160787,
       0.00700356, 0.0107855 , 0.0102984 , 0.00805875], dtype=float32)

In [ ]:
mean_list1 = test_pred1.mean().numpy().tolist()
std_list1 = test_pred1.stddev().numpy().tolist()
mean_list2 = test_pred2.mean().numpy().tolist()
std_list2 = test_pred2.stddev().numpy().tolist()
mean_list3 = test_pred3.mean().numpy().tolist()
std_list3 = test_pred3.stddev().numpy().tolist()

length = len(mean_list1)

kl_12 = [calculate_kl_divergence([mean_list1[k],std_list1[k]], [mean_list2[k],std_list2[k]]) for k in range(length)]
kl_13 = [calculate_kl_divergence([mean_list1[k],std_list1[k]], [mean_list3[k],std_list3[k]]) for k in range(length)]
# mean_distance12 = np.array(mean_list1) - np.array(mean_list2)
# mean_distance13 = np.array(mean_list1) - np.array(mean_list3)
qd_12 = [calculate_quantile_distance(test_pred4[k], test_pred5[k]) for k in range(length)]
qd_13 = [calculate_quantile_distance(test_pred4[k], test_pred6[k]) for k in range(length)]

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))
p = ax.twinx() #  

p.plot(input_list, kl_12, color='red', linestyle='--', label='KL divergence')
ax.plot(input_list, qd_12,color='blue', label='Quantile value difference')

ax.set_ylabel('Quantile value difference', fontsize=14, c='blue')
ax.set_xlabel('Wind speed (m/s)', fontsize=14)
p.set_ylabel('KL divergence', fontsize=14,  c='red')

ax.spines['left'].set_color('blue')
p.spines['left'].set_color('blue')
ax.spines['right'].set_color('red')
p.spines['right'].set_color('red')

p.tick_params(axis='y',labelsize=13,labelcolor='red',color='red')
ax.tick_params(axis='y',labelsize=13,labelcolor='blue',color='blue')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))
p = ax.twinx() # axis object that contains another y axis

p.plot(input_list, kl_13, color='red', linestyle='--', label='KL divergence')
ax.plot(input_list, qd_13,color='blue', label='Quantile value difference')

ax.set_ylabel('Quantile value difference', fontsize=14, c='blue')
ax.set_xlabel('Wind speed (m/s)', fontsize=14)
p.set_ylabel('KL divergence', fontsize=14,  c='red')

ax.spines['left'].set_color('blue')
p.spines['left'].set_color('blue')
ax.spines['right'].set_color('red')
p.spines['right'].set_color('red')

p.tick_params(axis='y',labelsize=13,labelcolor='red',color='red')
ax.tick_params(axis='y',labelsize=13,labelcolor='blue',color='blue')

plt.show()

# 9. Comparison with two baselines (Nw=4000)

In [7]:
import numpy as np
import pandas as pd

import math
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import tensorflow_probability as tfp
from keras.models import Sequential
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import StringLookup
import joblib
from tqdm import tqdm

import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model
from keras.utils.vis_utils import plot_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform,RandomNormal
from matplotlib import pyplot as plt

from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold, cross_val_score, train_test_split

tfd = tfp.distributions

#ignore warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [8]:
# brewer2mpl.get_map args: set name set type number of colors
import matplotlib as mpl

#mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

colors = ['#1E90FF', '#f12c60', '#fff21e', '#1efff2', '#1eafff', 'blueviolet', 'sandybrown', 'magenta', 'slategray']

In [9]:
from scipy.optimize import curve_fit

def f(x, a, b, c):
  # return 1 - np.exp(-(x/c)**k)
  return a*(x**3) + b*(x**2) + c*x

def find_nearest(a, a0):
    "Element in nd array `a` closest to the scalar value `a0`"
    idx = np.abs(a - a0).argmin()
    return idx

def BNN_normal():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x = Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  # outputs = Dense(1)(mean_x)
  model = keras.Model(inputs=[mean_inputs], outputs=[outputs])
  return model

In [11]:
wind_array = df.to_numpy()

# window length is set 4000
window = 4000
step = int(window/2)
# 5 = window/step
length = 1 + (len(wind_array) // window - 1)*2
nrm = [[] for j in range(length)]

for i in range(0, length):
    nrm[i] = wind_array[i*(step):i*(step) + window]

## 9.1 Plot the power curves for three methods (i=7)

In [ ]:
for q in range(23):
  if sum(label0[:,q])/4000 > 0.6:
    print(q)

13
19


In [ ]:
plt.scatter(speed[:,20], power[:,20])

In [ ]:
from scipy.stats import norm

i = 17
nrm0 = nrm[i]
speed0 = nrm0[:, 0::3]
power0 = nrm0[:, 1::3]
label0 = nrm0[:, 2::3]

speed = nrm0[:, 0::3]
power = nrm0[:, 1::3]
omega = 0.2
std = np.std(speed0[:,0])
speed_new = [[] for _ in range(23)]
power_new = [[] for _ in range(23)]
speed_new[0] = speed[:,0]
power_new[0] = power[:,0]

for j in range(1, 23):
  for t in range(len(speed[:,0])):
    idx = find_nearest(speed[:,j], speed[:,0][t])
    tmp = speed[:,j].flat[idx]
    if abs(tmp - speed[:,0][t]) <  omega*std:
      speed_new[j].append(tmp)
      power_new[j].append(power[:,j].flat[idx])

weight_list = np.load('baseline/WCDF_4000/%s.npy'%i)

m, n = 0, 0
fig, axes = plt.subplots(6, 4, figsize=(20, 25), dpi=300, sharex=True, sharey=True)

for k in range(23):
  print("Current segment: %s; WT: %s"%(i, k))
  speed_list = np.array(speed_new[k])
  power_list = np.array(power_new[k])
  power_list = (power_list - 0)/4000 # set between 0 and 1
  power_list[power_list>1] = 1
  power_list[power_list<0] = 0

  # save model weights
  speed_fit = []
  power_fit = []
  for l in range(len(speed_list)):
    if speed_list[l] >=4 and speed_list[l] <=12:
      speed_fit.append(speed_list[l])
      power_fit.append(power_list[l])

  axes[m][n].scatter(speed_fit, power_fit , s=3, c='darkgray', label = 'Actual data')
  model_gp = joblib.load('baseline/gp_4000/%s_%s.pkl'%(i,j))
  model_dpc = BNN_normal()
  model_dpc.load_weights("Nw4000/model_density/model_%s_%s.h5"%(i,k))

  input_list = np.linspace(4, 12, 10)

  step = 0
  for p in range(0, len(input_list), 2):
    test_pred = model_dpc(np.array(input_list[p]).reshape(1,1,1))

    prediction_mean = test_pred.mean().numpy().tolist()[0]
    prediction_stdv = test_pred.stddev().numpy()[0]

    prediction_mean = np.array(prediction_mean)
    prediction_mean[prediction_mean<0] = 0
    prediction_mean[prediction_mean>1] = 1

    x = np.linspace(prediction_mean-2*prediction_stdv, prediction_mean+2*prediction_stdv, 200)

    res = norm.pdf(x, loc=prediction_mean, scale=prediction_stdv)
    res = (res - min(res))/(1.5*(max(res) - min(res)))

    axes[m][n].plot(input_list[p]+res, x , linewidth=1.5, color=colors[step], label='Speed = %sm/s'%round(input_list[p], 2), linestyle='--')
    step += 1

  prediction = model_dpc(np.array(speed_fit).reshape(len(speed_fit), 1, 1))
  prediction_mean = prediction.mean().numpy()
  prediction_mean[prediction_mean<0] = 0
  prediction_mean[prediction_mean>1] = 1

  pred = f(np.array(speed_fit), weight_list[k][0], weight_list[k][1], weight_list[k][2])
  pred_gp = model_gp.predict(np.array(speed_fit).reshape(-1, 1), return_std=False)
  axes[m][n].scatter(speed_fit, prediction_mean, s=2, c='#f12c60', label='DPC')
  axes[m][n].scatter(speed_fit, pred, s=2, c='blue', label='LWZ')
  axes[m][n].scatter(speed_fit, pred_gp, s=2, c='orange', label='DKP')
  axes[m][n].set_ylim(-0.05,1.05)

  axes[m][n].set_title('WT #%s'%(k+1), fontsize=15)

  if n==0:
    axes[m][n].set_ylabel('Wind Power', fontsize=15)

  if m==5:
    axes[m][n].set_xlabel('Wind Speed', fontsize=15)

  n += 1
  if n >3:
    m += 1
    n = 0

axes[5][3].set_xlabel('Wind Speed', fontsize=15)
# axes[0][0].legend(bbox_to_anchor=(0, 1.2), loc=3, borderaxespad=0.1, ncol=5, fontsize=20)
lgd = axes[0][0].legend(bbox_to_anchor=(0, 1.2), loc=3, borderaxespad=0.1, ncol=5, fontsize=20)
for legend_handle in lgd.legendHandles:
  try:
    legend_handle.set_sizes([30])
  except:
    pass

In [ ]:
np.shape(speed_fit)

(3110,)

In [ ]:
pred = f(np.array(speed_fit), weight_list[k][0], weight_list[k][1], weight_list[k][2])

In [ ]:
pred

## 9.2 Control chart of three methods

### 9.1 GP

In [12]:
distance_matrix = np.load('baseline/gp_4000/distance_matrix.npy')

distance_matrix[np.isnan(distance_matrix)] = 50
distance_matrix[distance_matrix>50] = 50

In [ ]:
plt.hist(distance_matrix.flatten())

In [13]:
def calculate_cdf(dis, model, alpha):
  value_list = np.linspace(min(dis)-max(dis), 2*max(dis), 2000)

  for i in range(len(value_list)-1):
      value1 = model.integrate_box_1d(low = -np.Inf, high = value_list[i])
      value2 = model.integrate_box_1d(low = -np.Inf, high = value_list[i+1])
      if value1 <= alpha and value2 >= alpha:
          return (value_list[i] + value_list[i+1])/2

  return np.nan

In [14]:
from tqdm import tqdm
import scipy

ind = np.linspace(0.01,0.99,99).searchsorted(0.95)

UCL_list = []
for i in tqdm(range(23)):
  kde_model = scipy.stats.gaussian_kde(distance_matrix[i], bw_method = 0.5)
  cdf_list = [calculate_cdf(distance_matrix[i], kde_model, confidence) for confidence in np.linspace(0.01,0.99,99)]
  UCL_list.append(cdf_list[ind])

100%|██████████| 23/23 [01:38<00:00,  4.29s/it]


In [ ]:
len(distance_matrix[:,index_array[i]])

23

In [ ]:
from matplotlib.pyplot import MultipleLocator


marker_list = ['o', '^', 's', 'x', 'D']
color_list = ['#90EE90', 'magenta', 'sandybrown', '#1E90FF', 'darkorchid','gold', 'crimson']

fig, axes = plt.subplots(figsize=(12.3, 6), dpi=300)

index_list = [i for i in range(1, 24)]
# plt.plot(UCL_list, label='UCL of KDE', c='red', linestyle='--', linewidth=2,) # marker='s',markersize=5,
plt.plot(index_list, UCL_list, label='UCL of BCKDE', c='red', linewidth=2,) # marker='o', linestyle='--', markersize=5,

# for i in range(23):
#   color_id = i//5
#   marker_id = i%5
#   plt.plot(index_list, distance_matrix[:,i], label='WT #%s'%(i+1), c=color_list[color_id], marker=marker_list[marker_id], markersize=3, linewidth=1,linestyle='--')
index_array = np.array([6, 13, 15, 19, 20, 22])
for i in range(len(index_array)):
  color_id = i
  # color_id = i//5
  # marker_id = i%5
  plt.plot(index_list, distance_matrix[:,index_array[i]], label='WT #%s'%(index_array[i]+1), c=color_list[color_id], linewidth=1,linestyle='--')

plt.xlabel('Time Index', fontsize=16)
plt.ylabel('Scaled power difference', fontsize=16)
# plt.legend(bbox_to_anchor=(0, 1), loc=3, borderaxespad=0.1, ncol=7)

x_major_locator=MultipleLocator(1)
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.xlim(0.8,23.2)

In [ ]:
plt.figure(figsize=(8, 5))

plt.plot(UCL_list, label='UCL of BCKDE', c='red', linewidth=2,) # marker='o', linestyle='--', markersize=5,

distance_matrix[distance_matrix>1600] = 1600
for i in range(23):
  color_id = i//5
  marker_id = i%5
  plt.plot(distance_matrix[:,i], label='WT #%s'%(i+1), c=color_list[color_id], marker=marker_list[marker_id], markersize=3, linewidth=1,linestyle='--')

In [ ]:
predict_matrix = np.zeros((23, 23))

for i in range(23):
  for j in range(23):
    if distance_matrix[i][j] > UCL_list[i]:
      predict_matrix[i][j] = 1
    else:
      pass

degree_matrix = np.zeros((23, 23))

length = 4000

for i in tqdm(range(23)):
  nrm0 = nrm[i]
  label0 = nrm0[:, 2::3]
  degree_matrix[i] = np.sum(label0, axis=0)/length # 500 is the length of the data segment

degree_matrix[degree_matrix < 0.6] = 0
degree_matrix[degree_matrix >= 0.6] = 1
pred = predict_matrix.flatten()
raw = degree_matrix.flatten()

100%|██████████| 23/23 [00:00<00:00, 5227.82it/s]


In [ ]:
import seaborn as sns

sns.heatmap(degree_matrix, annot = False, cmap = 'coolwarm', linewidths = 0.1)

In [ ]:
TP, FP, FN = 0, 0, 0

for i in range(len(pred)):
    if pred[i] == 1 and raw[i] == 1:
        TP += 1
    elif pred[i] == 1 and raw[i] == 0:
        FP += 1
    elif pred[i] == 0 and raw[i] == 1:
        FN += 1
        print(int(i/23))
    else:
        pass

precision = TP/(TP + FP)
recall = TP/(TP + FN)

print('recall', recall)
print('precision', precision)
print('f1-score', (2*precision*recall)/(precision+recall))

7
10
recall 0.8666666666666667
precision 0.5416666666666666
f1-score 0.6666666666666667


In [ ]:
# for segment 7, WT #19
# for segment 10, WT #6
# for segment 12, WT #22
print(degree_matrix[12])

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


### 9.2 WCDF



In [16]:
from scipy.stats import f

predict_matrix = np.zeros((23, 23))

T2_list = []
UCL_list = []
for i in range(23):
  weight_list = np.load('baseline/WCDF_4000/%s.npy'%i)

  S = np.cov(weight_list.T)
  S_inv = np.linalg.inv(S)

  X_bar = np.mean(weight_list, axis=0)
  T2 = [np.array((np.matrix(weight_list[i]-X_bar)*np.matrix(S_inv)*np.matrix(weight_list[i] - X_bar).T))[0][0] for i in range(len(weight_list))]

  p = 3
  m = len(weight_list)
  alpha=0.05
  a, b=f.interval(1-alpha, dfn=p, dfd=m-p)

  UCL = (p*(m+1)*(m-1)*b)/(m**2-m*p)
  T2_list.append(T2)
  UCL_list.append(UCL)

  for k in range(23):
    if T2[k] > UCL:
      predict_matrix[i][k] = 1

In [17]:
T2_list = np.array(T2_list)

In [ ]:
from matplotlib.pyplot import MultipleLocator

fig, axes = plt.subplots(figsize=(12.3, 6), dpi=300)

marker_list = ['o', '^', 's', 'x', 'D']
color_list = ['#90EE90', 'magenta', 'sandybrown', '#1E90FF', 'darkorchid','gold', 'crimson']

index_list = [i for i in range(1, 24)]
plt.plot(index_list, UCL_list, label='UCL', c='red', linewidth=2,) # marker='o', linestyle='--', markersize=5,

index_array = np.array([6, 13, 15, 19, 20, 22])

for i in range(len(index_array)):
  color_id = i
  # color_id = i//5
  # marker_id = i%5
  plt.plot(index_list, T2_list[:,index_array[i]], label='WT #%s'%(index_array[i]+1), c=color_list[color_id], linewidth=1,linestyle='--')

plt.xlabel('Time Index', fontsize=16)
plt.ylabel('T2', fontsize=16)
# plt.legend(bbox_to_anchor=(0, 1), loc=3, borderaxespad=0.1, ncol=7)

x_major_locator=MultipleLocator(1)
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.xlim(0.8,23.2)

In [ ]:
degree_matrix = np.zeros((23, 23))

length = 4000

for i in tqdm(range(23)):
  nrm0 = nrm[i]
  label0 = nrm0[:, 2::3]
  degree_matrix[i] = np.sum(label0, axis=0)/length # 500 is the length of the data segment

degree_matrix[degree_matrix < 0.6] = 0
degree_matrix[degree_matrix >= 0.6] = 1
pred = predict_matrix.flatten()
raw = degree_matrix.flatten()

100%|██████████| 23/23 [00:00<00:00, 11596.22it/s]


In [ ]:
degree_matrix[12]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1.])

In [ ]:
# for segment 7, WT #13, 19
# for segment 10, WT #6
# for segment 12, WT #6, 22

TP, FP, FN = 0, 0, 0

for i in range(len(pred)):
    if pred[i] == 1 and raw[i] == 1:
        TP += 1
    elif pred[i] == 1 and raw[i] == 0:
        FP += 1
    elif pred[i] == 0 and raw[i] == 1:
        FN += 1
        print(int(i/23))
    else:
        pass

precision = TP/(TP + FP)
recall = TP/(TP + FN)

print('recall', recall)
print('precision', precision)
print('f1-score', (2*precision*recall)/(precision+recall))

7
7
8
10
11
12
12
recall 0.5333333333333333
precision 0.5714285714285714
f1-score 0.5517241379310344


### 9.3 DPC

In [19]:
distance_matrix = np.load('Nw4000/distance4dpc.npy',)

In [22]:
predict_matrix = pd.read_csv('Nw4000/bckde_dpc_0.8_624.csv').to_numpy()

In [ ]:
import seaborn as sns

sns.heatmap(predict_matrix, annot = False, cmap = 'coolwarm', linewidths = 0.1)

In [23]:
degree_matrix = np.zeros((23, 23))

length = 4000

for i in tqdm(range(23)):
  nrm0 = nrm[i]
  label0 = nrm0[:, 2::3]
  degree_matrix[i] = np.sum(label0, axis=0)/length # 500 is the length of the data segment

degree_matrix[degree_matrix < 0.6] = 0
degree_matrix[degree_matrix >= 0.6] = 1

pred = predict_matrix.flatten()
raw = degree_matrix.flatten()

100%|██████████| 23/23 [00:00<00:00, 3360.47it/s]


In [24]:
TP, FP, FN = 0, 0, 0

for i in range(len(pred)):
    if pred[i] == 1 and raw[i] == 1:
        TP += 1
    elif pred[i] == 1 and raw[i] == 0:
        FP += 1
    elif pred[i] == 0 and raw[i] == 1:
        FN += 1
    else:
        pass

precision = TP/(TP + FP)
recall = TP/(TP + FN)

print('recall', recall)
print('precision', precision)
print('f1-score', (2*precision*recall)/(precision+recall))

recall 0.7333333333333333
precision 0.9166666666666666
f1-score 0.8148148148148148


In [25]:
distance_matrix = np.load('Nw4000/distance4dpc.npy',)

In [ ]:
np.shape(distance_matrix)

(77, 23)

In [ ]:
df = pd.DataFrame()

distance_matrix = np.load('Nw4000/distance4dpc.npy',)
for i in range(23):
  df['%s'%i] = np.linspace(0, np.max(distance_matrix[i])+100, 1000)

df.to_csv('simulation4dpc_nw4000.csv', index=False)

In [26]:
# BCKDE
data4qpc = pd.read_csv('Nw4000/controlchart4dpc_nw4000.csv').to_numpy()
UCL_list_bckde = []
for i in tqdm(range(23)):
  ind = data4qpc[:,i].searchsorted(0.93)
  UCL_list_bckde.append(np.linspace(0, max(distance_matrix[i])+100, 1000)[ind])

100%|██████████| 23/23 [00:00<00:00, 5831.41it/s]


In [ ]:
import seaborn as sns

sns.heatmap(degree_matrix, annot = False, cmap = 'coolwarm', linewidths = 0.1)

In [ ]:
from matplotlib.pyplot import MultipleLocator

plt.figure(figsize=(8, 5))
marker_list = ['o', '^', 's', 'x', 'D']
color_list = ['#90EE90', 'magenta', 'sandybrown', '#1E90FF', 'darkorchid','gold', 'crimson']

fig, axes = plt.subplots(figsize=(12.3, 6), dpi=300)

index_list = [i for i in range(1, 24)]

# plt.plot(UCL_list, label='UCL of KDE', c='red', linestyle='--', linewidth=2,) # marker='s',markersize=5,
plt.plot(index_list, UCL_list_bckde, label='UCL of BCKDE', c='red', linewidth=2,) # marker='o', linestyle='--', markersize=5,

index_array = np.array([6, 13, 15, 19, 20, 22])
for i in range(len(index_array)):
  color_id = i
  # color_id = i//5
  # marker_id = i%5
  plt.plot(index_list, distance_matrix[:,index_array[i]], label='WT #%s'%(index_array[i]+1), c=color_list[color_id], linewidth=1,linestyle='--')

plt.xlabel('Time Index', fontsize=16)
plt.ylabel('Power curve distance', fontsize=16)
plt.legend(bbox_to_anchor=(0, 1), loc=3, borderaxespad=0.1, ncol=8)

x_major_locator=MultipleLocator(1)
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.xlim(0.8, 23.2)

In [ ]:
# for segment 7, WT #13, 19
# for segment 10, WT #6
# for segment 12, WT #6, 22

In [ ]:
predict_matrix = np.zeros((23, 23))

for i in range(23):
  for j in range(23):
    if distance_matrix[i][j] > UCL_list_bckde[i]:
      predict_matrix[i][j] = 1
    else:
      pass

degree_matrix = np.zeros((23, 23))

length = 4000

for i in tqdm(range(23)):
  nrm0 = nrm[i]
  label0 = nrm0[:, 2::3]
  degree_matrix[i] = np.sum(label0, axis=0)/length # 500 is the length of the data segment

degree_matrix[degree_matrix < 0.6] = 0
degree_matrix[degree_matrix >= 0.6] = 1
pred = predict_matrix.flatten()
raw = degree_matrix.flatten()

100%|██████████| 23/23 [00:00<00:00, 5470.93it/s]


In [ ]:
TP, FP, FN = 0, 0, 0

for i in range(len(pred)):
    if pred[i] == 1 and raw[i] == 1:
        TP += 1
    elif pred[i] == 1 and raw[i] == 0:
        FP += 1
    elif pred[i] == 0 and raw[i] == 1:
        FN += 1
    else:
        pass

precision = TP/(TP + FP)
recall = TP/(TP + FN)

print('recall', recall)
print('precision', precision)
print('f1-score', (2*precision*recall)/(precision+recall))

recall 0.8
precision 0.9230769230769231
f1-score 0.8571428571428571
